In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
cs_file = '../data/Mass_balance.tab'
dfmb = pd.read_csv(cs_file, delimiter='\t')
dfmb.rename(columns = {'Event':'Stake', 'Date/Time':'Date', 
                      'Elevation [m]':'Elevation', 'Diff':'SMB'}, inplace=True)
dfmb['Date'] = pd.to_datetime(dfmb['Date'])

In [3]:
df_s = dfmb.sort_values(by=['Elevation', 'Date'])
df_s.index = df_s.Date
df_s['Year'] = df_s.index.year

In [4]:
df_s

,Stake,Date,Latitude,Longitude,Elevation,SMB,Year
Date,,,,,,,
2010-11-22,PG19,2010-11-22,-62.237672,-58.633624,96.8,2.12,2010
2010-12-05,PG19,2010-12-05,-62.237672,-58.633624,96.8,0.13,2010
2010-12-11,PG19,2010-12-11,-62.237672,-58.633624,96.8,0.20,2010
2010-12-23,PG19,2010-12-23,-62.237672,-58.633624,96.8,-0.21,2010
2011-01-09,PG19,2011-01-09,-62.237672,-58.633624,96.8,-0.59,2011
...,...,...,...,...,...,...,...
2012-01-27,PG11,2012-01-27,-62.199141,-58.581200,465.6,2.19,2012
2013-02-16,PG11,2013-02-16,-62.199141,-58.581200,465.6,-0.01,2013
2013-02-19,PG11,2013-02-19,-62.199141,-58.581200,465.6,-0.01,2013


In [5]:
labels = df_s.drop_duplicates(subset=['Stake'])

In [7]:
label_year = df_s.drop_duplicates(subset=['Year']).sort_values(by=['Year'])['Year'].values

In [8]:
label_year

array([2010, 2011, 2012, 2013, 2014, 2015, 2016])

In [91]:
smb_sea = []
for j in labels['Stake'][0:10]:
    stake = df_s.loc[lambda df: df['Stake'] == j, :]
    for i in label_year[:-1]:
        # SMB winter
        time1_w = str(i)+'0501'
        time2_w = str(i)+'1130'
        subset_w = stake[time1_w:time2_w]
        if len(subset_w) > 1:
            subset_w1 = subset_w[-1:]
            subset_w1['SMB'] = subset_w['SMB'].cumsum()[-1]
            subset_w = subset_w1.copy()
        smb_sea.append(subset_w)
        # SMB summer
        time1_s = str(i)+'1201'
        time2_s = str(i+1)+'0430'
        subset_s =stake[time1_s:time2_s]
        if len(subset_s) > 1:
            subset_s1 = subset_s[-1:]
            subset_s1['SMB'] = subset_s['SMB'].cumsum()[-1]
            subset_s = subset_s1.copy()
        smb_sea.append(subset_s)


In [95]:
smb_sea_t = pd.concat(smb_sea)
smb_sea_t.reset_index(drop=True, inplace=True)

In [100]:
smb_sea_t0 = []
for j in labels['Stake'][0:10]:
    stake = smb_sea_t.loc[lambda df: df['Stake'] == j, :].copy()
    date0 = stake[0:1].copy()
    date0['Date'] = np.datetime64('2010-05-01')
    smb_sea_t0.append(date0)
    smb_sea_t0.append(stake)

In [101]:
smb_sea_t = pd.concat(smb_sea_t0)

In [103]:
date_1 = []
for j in labels['Stake'][0:10]:    
    stake = smb_sea_t.loc[lambda df: df['Stake'] == j, :].copy()
    date_1.append(stake['Date'].values[0])
    for i in stake.Date[0:-1]:
        date_1.append(i)

In [104]:
smb_sea_t['Date1'] = date_1

In [105]:
smb_sea_t['Days'] = (smb_sea_t['Date'] - smb_sea_t['Date1']).dt.days

In [106]:
smb_coll_df1 = smb_sea_t[['Stake', 'Date', 'Date1', 'Latitude', 'Longitude', 
                          'Elevation', 'Days','SMB']]

In [109]:
smb_coll_df1[0:20]

,Stake,Date,Date1,Latitude,Longitude,Elevation,Days,SMB
0,PG19,2010-05-01,2010-05-01,-62.237672,-58.633624,96.8,0,2.12
0,PG19,2010-11-22,2010-05-01,-62.237672,-58.633624,96.8,205,2.12
1,PG19,2011-04-06,2010-11-22,-62.237672,-58.633624,96.8,135,-1.20
2,PG19,2011-10-15,2011-04-06,-62.237672,-58.633624,96.8,192,0.83
3,PG19,2012-04-21,2011-10-15,-62.237672,-58.633624,96.8,189,-3.45
4,PG19,2012-11-24,2012-04-21,-62.237672,-58.633624,96.8,217,2.50
5,PG19,2013-04-11,2012-11-24,-62.237672,-58.633624,96.8,138,-2.00
6,PG19,2013-11-26,2013-04-11,-62.237672,-58.633624,96.8,229,1.36
7,PG19,2014-04-30,2013-11-26,-62.237672,-58.633624,96.8,155,-0.67
8,PG19,2014-11-30,2014-04-30,-62.237672,-58.633624,96.8,214,1.85


In [108]:
smb_coll_df1.to_csv('../data/SMB_four_season.csv', index=False, sep='\t')